# Web Scraping - Collecting Data about Monsters - Working Notebook

## 1. First Steps: Using Beautiful Soup to Scrape a Wikipedia Page

We are going to scrape Wikipedia for information about various mythological creatures. The information we want exists in multiple layers, like a tree structure. We're starting from [this list](https://en.wikipedia.org/wiki/Lists_of_legendary_creatures). This is a top level page which is a list of lists, one for each letter (eg `List of legendary creatures (A)`). The `A` sublist contains a list of creatures, as well as each creature's cultural origin. We can go one level deeper to the creature itself for even more information.
- Lists of legendary creatures
    - List of legendary creatures (A)
    - List of legendary creatures (B)
        - Ba (Egyptian)
        - Baba Yaga (Slavic)
        - Backoo (Guyanese) ...
    - List of legendary creatures (C) ...

Let's grab the list of lists and try to extract our sublists. We'll use the Requests library, as we did with APIs, as well as [Beautiful Soup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/#), which is a library for extracting data from HTML and XML. We're going to use the `html5lib` parser with Beautiful Soup because it is pretty lenient and creates valid HTML; the downside is that it is slightly slower than some other parsers.

So, first, let's import all the modules we need, including Beautiful Soup, which we'll simply call `bs`.

In [ ]:
import requests 
from bs4 import BeautifulSoup as bs
import string
import re

Now that we have our modules, the following cell will scrape our list of lists of legendary creatures.

As with our API queries, the line of script that uses `requests` to do the scraping is very straightforward: `R = requests.get(list_of_lists_url)`.

Once we have that HTML content, we'll then parse it into a tree structure of Python objects using Beautiful Soup.

In [ ]:
list_of_lists_url = "https://en.wikipedia.org/wiki/Lists_of_legendary_creatures"
R = requests.get(list_of_lists_url)
soup = bs(R.content, 'html5lib')

Let's take a quick look at the massive blob of HTML that we've just scraped.

In [ ]:
soup

That's a little unweildly. Let's instead see a "pretty" version of the HTML tree using the `soup.prettify()` method. Once you run the following cell, the HTML will be structured in a much more human-readable format.

In [ ]:
print(soup.prettify())

If, instead, we print the actual raw content using `print(R.content)`, we can see that that it looks like one long string.

In [ ]:
print(R.content)

This is what we downloaded without any formatting. It's actually a sequence of <b>byte literals</b>. You can tell this because byte literals are prefixed by `b` when printed.

So far, so good.

## 2. Navigating HTML with Beautiful Soup

There's a lot that you can do with the HTML data that you've just scraped, but to do so you have to navigate this mass of information effectively. Fortunately, Beautiful Soup includes a number of methods that are built for just this purpose. Most of these methods isolate specific HTML tags, like `<title>` or `<body>` or `<p>`. We'll look at a few here, but you can always [consult the documentation](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) if you want to see a full list.

First of all, we can isolate the text from the page without any HTML markup:

In [ ]:
print(soup.get_text())

This looks a little messy at the top of the page, but if you scroll down, you'll see that a good deal of the text is captured quite cleanly.

Similarly, you can isolate the `title` element. By using the `title` method by itself, you'll retain the HTML tags for this element.

In [ ]:
soup.title

Or, if you prefer, you can remove those tags by isolating the string itself.

In [ ]:
print(soup.title.string)

Here, we get the first paragraph element, using `.p`

In [ ]:
soup.p

Feel free to modify the cell above to locate other common HTML elements, like `.head`, `.body`, `.div` and `.a`.

Often, you'll want to search for a specific occurence of a tag using attributes such as a classname or id. You can use the `.find()` method to specify these attributes. Let's grab the Table of Contents from our soup. In a browser window, open up [the Wikipedia page]('https://en.wikipedia.org/wiki/Lists_of_legendary_creatures'). Right click on the table of contents and choose "Inspect" (in Chrome) or "Inspect Element" (Firefox). This will show the current state of the DOM and open to the point in the DOM or "Document Object Model" where you've clicked. You should see that there is a `<div>` with an `id` of `"toc"`. Let's select that with BeautifulSoup:

In [ ]:
tag = soup.find("div", id="toc")
print(tag)

It still looks a bit complicated, but we've cleanly isolated the Table of Contents for this particular page.

Now that we've isolated this one `div` tag, we can see all sorts of information about it, including its type, name, attributes, and id.

In [ ]:
print(type(tag))
print(tag.name)
print(tag.attrs) 
print(tag['id'])

We could have just used `soup.find(id="toc")` because HTML IDs should be unique, but sometimes you'll want to combine selector attributes as we did here. This shows that BeautifulSoup tags are their own Python class, and have names and attributes. Those attributes can be accessed just like you would access a dictionary property.

## 3. Compiling the Alphabetical Lists of Legendary Creatures

As we've seen, in order to compile a master list of legendary creatures, we'll have to scrape each of the alphabetical lists sequentially: all the creatures that start with "A" (like, "Anubis"), all that start with "B" (like, "Basilisk"), etc. To do that, we'll need the corresponding URL for each of the alphebetical lists.

Let's start by trying to get every instance of the `a` or `anchor` tag, which marks each link on our page. To do this, we'll use the `.find_all()` method.

In [ ]:
all_links = soup.find_all('a')
print(len(all_links))
all_links

Whoa, that's way too many links! <b>(You'll probably see a different number from what Cole sees in the video.)</b> We need to find just the hyperlinks for the alphabetical lists. Go ahead and try finding one of the links in the developer's console and seeing if there are any special attributes that might allow us to easily select it.

To make it easy for you, we've included this little snippet of our pretified HTML, so you can see the beginning of the list of links that concerns us.

![Prettified HTML](alphabetical_lists_-_legendary_creatures.png "Alphabetical Lists - Legendary Creatures")

Unfortunately, it looks like there's no class for the links we want, nor is there an id or class on the containing list element (`<ul>`). There is an ID in the span of the header above the list element we want (`Alphabetical_lists`). We'll need to do some slightly finicky traversing of the tree to get the data: first, find the span; then go from the span to the header (`parent`); get the first member of the parent's `siblings`; and finally, get all of the links in that unordered list (`<li>` tags in the `<ul>`). You can do this using `.contents`, which returns a list, or `.children`, which returns an iterator.

Let's find the `span` with the `id` "Alphabetical_lists` first.

In [ ]:
tag_header = soup.find(id="Alphabetical_lists")
print(tag_header)

Now, navigating through this soup of HTML can be a little tricky, but beautifulsoup fortunately has a number of methods that can be used for just this purpose:

    `parent` - moves from one element in the HTML to another that contains it
    `children` - which does the opposite, moves from an an element to those that it contains
    `next_sibling` - moves between two elements on the same level
    
Follow along as Cole works his way through the HTML to find the elements we need.
    

In [ ]:
tag_parent = tag_header.parent
print(tag_parent)

In [ ]:
tag_nextsib = tag_parent.next_sibling
print(tag_nextsib)

As Cole points out, `next_sibling` will sometimes give you empty blank space. Instead, use `find_next_sibling` to proceed right to the unordered list that we want.

In [ ]:
tag_ul = tag_parent.find_next_sibling()
print(tag_ul)

There's one last step. Now that we have this list, we have to be able to isolate each subdirectory, first by singling out one HTML link, and then using `.get("href")` to extract the subdirectory URL. It works like this:

In [ ]:
link1 = tag_ul.find("a")

In [ ]:
link1.get("href")

Now that you've isolated both our unordered list and the subdirectory URL within each link, you should be able to use what you know to produce a list of each of the complete alphabetical URLs.

On your own, take a stab at writing a loop that 1) singles out the list element for each letter, 2) identifies each URL subdirectory, 3) appends that subdirectory to the main wikipedia URL , which is `"https://en.wikipedia.org"`, and 4) finally, make a list of each of these complete URLs.

Here, we've given you a head start with some hints:

In [ ]:
hrefs =                   # Create an empty list

for  in :                 # Isolate each link using `.find_all`. Should take the form `for X in Y:`
    url =      +          # Using `.get("href")`, isolate each subdirectory URL and add it to the main wikipedia URL
                          # Append the full URL to your list
print(hrefs)

Pause the video here and complete the loop!

## 4. Scraping List Pages and Creating a Dictionary of Creatures

At this point, you've learned how to scrape a webpage and parse its HTML to find the elements that you need. That was our main objective. For the sake of completeness, we'll show you how Cole went on to scrape a much more complete database, with all sorts of information about these legendary creatures. Much of this is pretty finicky, so we're going to move much more quickly, and we won't be walking through each step in the process.

First, we're going to compile all the creatures into one source. Let's make a dictionary for that, using creature names as keys. Then we'll want to create a function that we can reuse to scrape each of the list pages for the creatures. There are several pages that have some irregular items without links or the dashes we're using to separate the creature names from their short descriptions.

Here's the dictionary:

In [ ]:
all_creatures = {}

And here is the function Cole wrote to populate his dictionary. If this looks complicated, much of the code here is involved in cleaning up the names and short descriptions for each creature.

In [ ]:
def scrape_list_page(url):
    """
    Scrapes a list of creature pages.
    Returns a dictionary of creatures with names, titles (of links), links, short descriptions, and cultures.
    """
    
    creatures = {}
    R = requests.get(url)
    soup = bs(R.content, 'html5lib')
    list_items = soup.find("div", class_="mw-parser-output").find_all("ul")[1].find_all("li")
    for li in list_items:
        if len(li.find_all("a")) < 1: # There are a couple items without links
            split_text = li.get_text().split("-", 1)[0].strip()
            name = split_text[0].strip()
            desc = split_text[1].strip()
            creatures[name] = {
                "name": name,
                "short_description": desc
            }
        else:
            name = li.a.contents[0]
            creatures[name] = {}
            creatures[name]["name"] = name
            creatures[name]["title"] = li.a['title']
            creatures[name]["link"] = requests.compat.urljoin("https://en.wikipedia.org/wiki", li.a['href'])
            if "-" in li.get_text(): # There are a couple items where the dash isn't surrounded by spaces
                creatures[name]["short_description"] = li.get_text().split("-", 1)[1].strip()
            if(len(li.find_all("a")) > 1): # Couple of items without links
                creatures[name]["culture"] = li.find_all("a")[1].contents[0]
                creatures[name]["culture_title"] = li.find_all("a")[1]['title']
                creatures[name]["culture_href"] = requests.compat.urljoin("https://en.wikipedia.org/wiki", li.find_all("a")[1]['href'])
    return creatures

Let's test this on one page of our `hrefs` list to make sure it works. If that looks good, then we can run it on all the subpages.

In [ ]:
print(hrefs[0])
all_creatures.update(scrape_list_page(hrefs[0]))
all_creatures

Beautiful. Now that we have that information, let's run a loop that collects data from each of the alphabetical URLs we collected.

In [ ]:
for creature_list_url in hrefs:
    creatures = scrape_list_page(creature_list_url)
    all_creatures.update(creatures)
print(len(all_creatures.keys()))

Now that we have our complete list of creatures, it's a simple matter to print the information for each of them.

In [ ]:
all_creatures["Vampire"]

### Exporting Our Creatures

Great! That's a lot of data. Let's export it all to a CSV in case we want to import it again later. We could use the `csv` library, but Pandas has a quick handy function for this as well.

In [ ]:
import pandas as pd
import csv
creatures_list = []

In [ ]:
for key, creature in all_creatures.items():
    creatures_list.append(creature)
df = pd.DataFrame(creatures_list)
df.to_csv("all_creatures.csv", header=True, index=False)

We've made our CSV and the job is done! Find the csv in your section 2 directory and check in out!